In [2]:
import cv2
import numpy as np
import os
from IPython.display import display, clear_output
from PIL import Image

# Paths (update to your actual paths)
haar_file = r'C:\Users\avina\OneDrive\Documents\ai-30days\day-7\Code\haarcascade_frontalface_default.xml'
datasets = r'C:\Users\avina\OneDrive\Documents\ai-30days\day-7\Code\datasets'
(width, height) = (130, 100)

# Create dataset folder if it doesn't exist
if not os.path.exists(datasets):
    os.makedirs(datasets)


def collect_data(user_name):
    path = os.path.join(datasets, user_name)
    if not os.path.isdir(path):
        os.mkdir(path)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)
    count = 1
    print(f"[INFO] Collecting data for '{user_name}'...")

    try:
        while count <= 100:
            ret, im = webcam.read()
            if not ret:
                print("[ERROR] Failed to capture image from webcam.")
                break
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 4)

            for (x, y, w, h) in faces:
                cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
                face = gray[y:y + h, x:x + w]
                face_resize = cv2.resize(face, (width, height))
                cv2.imwrite(f'{path}/{count}.png', face_resize)
                count += 1

            # Convert BGR to RGB for displaying in Jupyter
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pil_im = Image.fromarray(im_rgb)

            clear_output(wait=True)
            display(pil_im)
            print(f"Capturing image {count-1}/100")

    except KeyboardInterrupt:
        print("[INFO] Data collection interrupted by user.")

    webcam.release()
    clear_output()
    print(f"[INFO] Dataset collection completed for '{user_name}'.")


def recognize_faces():
    print("[INFO] Training recognizer...")
    images, labels, names, id = [], [], {}, 0

    for subdir in os.listdir(datasets):
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = os.path.join(subjectpath, filename)
            images.append(cv2.imread(path, 0))
            labels.append(id)
        id += 1

    images, labels = [np.array(lis) for lis in [images, labels]]

    model = cv2.face.FisherFaceRecognizer_create()
    model.train(images, labels)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)
    cnt = 0
    print("[INFO] Starting face recognition... Press Ctrl+C to exit.")

    try:
        while True:
            ret, im = webcam.read()
            if not ret:
                print("[ERROR] Failed to capture image from webcam.")
                break
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face = gray[y:y + h, x:x + w]
                face_resize = cv2.resize(face, (width, height))
                prediction = model.predict(face_resize)

                cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)

                if prediction[1] < 800:
                    cv2.putText(im, f'{names[prediction[0]]} - {int(prediction[1])}', (x - 10, y - 10),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (51, 255, 255))
                    print(f"[INFO] Recognized: {names[prediction[0]]}")
                    cnt = 0
                else:
                    cnt += 1
                    cv2.putText(im, 'Unknown', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                    if cnt > 100:
                        print("[WARNING] Unknown person detected!")
                        cv2.imwrite("input.jpg", im)
                        cnt = 0

            # Convert BGR to RGB for displaying in Jupyter
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            pil_im = Image.fromarray(im_rgb)

            clear_output(wait=True)
            display(pil_im)

    except KeyboardInterrupt:
        print("[INFO] Face recognition stopped by user.")

    webcam.release()
    clear_output()


# Main menu for notebook
print("========== Face Recognition System ==========")
print("1. Collect Face Data")
print("2. Recognize Faces")
choice = input("Enter choice (1/2): ").strip()

if choice == '1':
    username = input("Enter your name: ").strip()
    collect_data(username)
elif choice == '2':
    recognize_faces()
else:
    print("Invalid choice. Exiting...")
